In [21]:
import igraph

In [22]:
print(igraph.__version__)

0.7.1


In [24]:
g = igraph.Graph()

In [25]:
g

In [26]:
print(g)

IGRAPH U--- 0 0 --


In [27]:
g.add_vertices(3)

In [28]:
g

In [29]:
print(g)

IGRAPH U--- 3 0 --


In [30]:
g.add_edges([(0,1),(1,2)])

In [31]:
print(g)

IGRAPH U--- 3 2 --
+ edges:
0--1 1--2


In [34]:
g.add_edges([(2,0)])

In [35]:
g.add_vertices(3)

In [36]:
print(g)

IGRAPH U--- 6 3 --
+ edges:
0--1 1--2 0--2


In [37]:
layout = g.layout("kk")
plot(g, layout = layout)

NameError: name 'plot' is not defined